# Project VINF FIIT
#### Creating a dictionary of pairs for the purposes of Named Entity Recognizing: Wiki page - type

Regular expresion to extract articles
or
by this approach of using SAX XML parser<

## 1. Part : Downloading Wikipedia articles


In [2]:
import requests
from bs4 import BeautifulSoup
import os
import re

In [4]:
base_url = 'https://dumps.wikimedia.org/enwiki/20201001/'
base_html = requests.get(base_url).text
base_html[:15]

'<!DOCTYPE html '

In [8]:
soup_dump = BeautifulSoup(base_html, 'html.parser')
soup_dump.find_all('li', {'class': 'file'}, limit = 10)[0]

<li class="file"><a href="/enwiki/20201001/enwiki-20201001-pages-articles-multistream.xml.bz2">enwiki-20201001-pages-articles-multistream.xml.bz2</a> 17.5 GB</li>

In [9]:
files = []
for file in soup_dump.find_all('li', {'class': 'file'}):
    text = file.text
    if 'pages-articles' in text:
        files.append((text.split()[0], text.split()[1:]))
files[:5]

[('enwiki-20201001-pages-articles-multistream.xml.bz2', ['17.5', 'GB']),
 ('enwiki-20201001-pages-articles-multistream-index.txt.bz2', ['215.8', 'MB']),
 ('enwiki-20201001-pages-articles-multistream1.xml-p1p41242.bz2',
  ['231.7', 'MB']),
 ('enwiki-20201001-pages-articles-multistream-index1.txt-p1p41242.bz2',
  ['222', 'KB']),
 ('enwiki-20201001-pages-articles-multistream2.xml-p41243p151573.bz2',
  ['313.2', 'MB'])]

In [63]:
files_to_download = [file[0] for file in files if re.search('pages-articles\d{1,2}.xml-p',file[0])]
files_to_download

['enwiki-20201001-pages-articles1.xml-p1p41242.bz2',
 'enwiki-20201001-pages-articles2.xml-p41243p151573.bz2',
 'enwiki-20201001-pages-articles3.xml-p151574p311329.bz2',
 'enwiki-20201001-pages-articles4.xml-p311330p558391.bz2',
 'enwiki-20201001-pages-articles5.xml-p558392p958045.bz2',
 'enwiki-20201001-pages-articles6.xml-p958046p1483661.bz2',
 'enwiki-20201001-pages-articles7.xml-p1483662p2134111.bz2',
 'enwiki-20201001-pages-articles8.xml-p2134112p2936260.bz2',
 'enwiki-20201001-pages-articles9.xml-p2936261p4045402.bz2',
 'enwiki-20201001-pages-articles10.xml-p4045403p5399366.bz2',
 'enwiki-20201001-pages-articles11.xml-p5399367p6899366.bz2',
 'enwiki-20201001-pages-articles11.xml-p6899367p7054859.bz2',
 'enwiki-20201001-pages-articles12.xml-p7054860p8554859.bz2',
 'enwiki-20201001-pages-articles12.xml-p8554860p9172788.bz2',
 'enwiki-20201001-pages-articles13.xml-p9172789p10672788.bz2',
 'enwiki-20201001-pages-articles13.xml-p10672789p11659682.bz2',
 'enwiki-20201001-pages-articles

Using keras method get_file for downloading wanted files

In [70]:
import sys
from keras.utils import get_file
directory = '/home/xminarikd/.keras/datasets/'

In [74]:
data_paths = []
file_info = []

for file in files_to_download:
    path = directory + file
    
    if not os.path.exists(directory):
        print('neexistuje')
    # downaload only when file dont exist
    if not os.path.exists(directory + file):
        print('Downloading')
        # If not, download the file
        data_paths.append(get_file(file, base_url + file))
        # Find the file size in MB
        file_size = os.stat(path).st_size / 1e6
        
        # Find the number of articles
        file_articles = int(file.split('p')[-1].split('.')[-2]) - int(file.split('p')[-2])
        file_info.append((file, file_size, file_articles))
        
    # when file already exist
    else:
        data_paths.append(path)
        file_size = os.stat(path).st_size / 1e6
        
        file_number = int(file.split('p')[-1].split('.')[-2]) - int(file.split('p')[-2])
        file_info.append((file.split('-')[-1], file_size, file_number))

Downloading
242098176/242093817 [==============================] - 330s 1us/step
Downloading
324780032/324777650 [==============================] - 445s 1us/step
Downloading
352124928/352119906 [==============================] - 313s 1us/step
Downloading
389988352/389987127 [==============================] - 372s 1us/step
Downloading
420814848/420806959 [==============================] - 471s 1us/step
Downloading
450748416/450745879 [==============================] - 553s 1us/step
Downloading
462512128/462504094 [==============================] - 385s 1us/step
Downloading
471654400/471652241 [==============================] - 344s 1us/step
Downloading
512147456/512145263 [==============================] - 485s 1us/step
Downloading
502456320/502449820 [==============================] - 357s 1us/step
Downloading
486776832/486770345 [==============================] - 393s 1us/step
Downloading
46866432/46859027 [==============================] - 69s 1us/step
Downloading
404406272/404401272

In [76]:
sorted(file_info, key = lambda x: x[1], reverse = True)[:5]

[('enwiki-20201001-pages-articles9.xml-p2936261p4045402.bz2',
  512.145263,
  1109141),
 ('enwiki-20201001-pages-articles10.xml-p4045403p5399366.bz2',
  502.44982,
  1353963),
 ('enwiki-20201001-pages-articles11.xml-p5399367p6899366.bz2',
  486.770345,
  1499999),
 ('enwiki-20201001-pages-articles8.xml-p2134112p2936260.bz2',
  471.652241,
  802148),
 ('enwiki-20201001-pages-articles7.xml-p1483662p2134111.bz2',
  462.504094,
  650449)]

## 2. Part Parsing data

In [10]:
import subprocess
import xml.sax
import regex
import pandas as pd

In [12]:
#Get Infobox and Infobox type from article text

#source: https://regex101.com/
exp_infobox = '(?=\{Infobox)(\{([^{}]|(?1))*\})'
exp_inf_type = '(?<=Infobox)(.*?)(?=(\||\n|<!-|<--))'

#doplnit mozno osetrenie redirect
def getInfType(text):
    infobox = regex.search(exp_infobox, text).group() if regex.search(exp_infobox, text) is not None else None
    if infobox is not None:
        return regex.search(exp_inf_type, infobox).group().strip() if regex.search(exp_inf_type, infobox) is not None else None

In [11]:
#docs: https://docs.python.org/3.8/library/xml.sax.handler.html
class ContentHandler(xml.sax.handler.ContentHandler):
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buf = None
        self._parts = {}
        self._last_tag = None
        self.output = []

    def characters(self, content):
        if self._last_tag:
            self._buf.append(content)

    def startElement(self, name, attrs):
        if name == 'page':
            self._parts = {}
        if name in ('title', 'text'):
            self._last_tag = name
            self._buf = []

    def endElement(self, name):
        if name == self._last_tag:
            self._parts[name] = ''.join(self._buf)
        
        #whole article
        if name == 'page':
            self.output.append((self._parts['title'], getInfType(self._parts['text']) , self._parts['text']))
            #self.output.append((self._parts['title'], getInfType(self._parts['text'])))

In [24]:
test_data_path = '/home/xminarikd/.keras/datasets/enwiki-20201001-pages-articles9.xml-p2936261p4045402.bz2'
handler = ContentHandler()

parser = xml.sax.make_parser()
parser.setContentHandler(handler)

for line in subprocess.Popen(['bzcat'], 
                         stdin = open(test_data_path), 
                         stdout = subprocess.PIPE).stdout:
    parser.feed(line)
    
    # get only some results
    if len(handler.output) > 10000:
        break
        
for x in handler.output:
    print(x[0])

#print(handler.output[10][0])
#print(regex.search(exp_inf_type, infobox).group().strip())

David Stagg
Amaranthus mantegazzianus
Amaranthus quitensis
KTXT
Maud Queen of Norway
Milligram per litre
Utica Psychiatric Center
Wikipedia:Articles for deletion/Studiomuscle
Olean Wholesale Grocery
Queen Tiye
Queen Hatshepsut
Clibanarii
File:Hanns Martin Schleyer in captivity.jpg
Political documentary
Wikipedia:Articles for deletion/Slarp
Final fantasy legends
Queen Marie Amelie Therese
Political documentaries
E-767
Wikipedia:Articles for deletion/"dirty thirty"
Prince Edward-Lennox
Arthur Hill (actor)
Periodic paralysis
Greenstripe
Amaranthus cruentus
Careless weed
Zamil idris
Khada sag
Million instructions per second
Ashtadiggajas
John C.Harsanyi
Société entomologique de France
Sangorache
Joseph's coat
Recipients of the Distinguished Service Award of the Order of the Arrow
Josephscoat
Joseph's-coat
URM Stores
General purpose technology
Rough skinned newt
Queen Sophie Dorothea of Hanover
Cefepime
Larry Zerner
World of Illushions Starring Mickey Mouse and Donald Duck
Category:Aberdeen

Were-lions
Category:Schools in Ottawa
Tymczasowy Rząd Jedności Narodowej
Rząd Tymczasowy Rzeczypospolitej Polskiej
Waddesdon Church of England School
Matrix logarithm
Aria Viscairough
Kate Tucker
File:Live at the Isle of Wight Festival 1970.jpg
William Edwin Melton
File:New Buildings at Dulwich College. ILN. 1869.jpg
Catherine Micaela
Ikey Soloman
Bidding-prayer
Fagen
1955–56 European Rugby League Championship
Al-Dujail trial
List of Nonesuch Records artists
South East Dorset conurbation
Burnous
The clock of the long now
O. Madhavan
Westbourne, West Sussex
Westbourne, London
XWiki.com
Westbourne, Dorset
FOK
Theveste
Emmanuel College, Toronto
Renault Alpine GTA/A610
I Am All of Me
Eurocodes
Ann Risley
Manolo
Japanese cruiser Yoshino
Gasoline theft
Regis College, Toronto
1969–70 European Rugby League Championship
List of northernmost items
Wikipedia:Articles for deletion/Smash my ipod
Graham Weir
File:RonaldWelchKnightCrusader1.jpg
File:New Charterhouse School Buildings. ILN 1872.jpg
Jen

Madison Administration
J. Q. Adams Administration
Philippe Blasband
John Quincy Adams Administration
House of Representatives (Belize)
HMGS-South
J. Q. Adams
Duisburg Philharmonic
East Claydon School
Jackson Administration
Michael Ettmüller
Ballardvale, massachusetts
1977–78 NHL season
Van Buren Administration
State transition diagram
Capital Levy
Special Class Railway Apprentice
Category:Royal Australian Navy sailors
Till the End of Time
W. Harrison Administration
William Warren Scranton
W. H. Harrison Administration
William W. Scranton III
Loyalist Association of Workers
William Harrison Administration
Till The End Of Time (1991 song)
Hugh Iorys Hughes
Taisekiji temple
William Henry Harrison Administration
Devil's grandmother
William H. Harrison Administration
Edlesborough School
Frederick William Freking
Taylor Administration
Convertible Debenture
Till The End Of Time (1945 song)
Tyler Administration
Fan (enthusiast)
Sympathetic nervous sytem
Boughton House
Du Maurier (cigarette)
Li

### Test and data searching area

In [14]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn

In [20]:
for i, line in enumerate(subprocess.Popen(['bzcat'], 
                              stdin = open(test_data_path), 
                              stdout = subprocess.PIPE).stdout):
    print(i,'<==>', line)
    if i > 300:
        break

0 <==> b'<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.10/ http://www.mediawiki.org/xml/export-0.10.xsd" version="0.10" xml:lang="en">\n'
1 <==> b'  <siteinfo>\n'
2 <==> b'    <sitename>Wikipedia</sitename>\n'
3 <==> b'    <dbname>enwiki</dbname>\n'
4 <==> b'    <base>https://en.wikipedia.org/wiki/Main_Page</base>\n'
5 <==> b'    <generator>MediaWiki 1.36.0-wmf.10</generator>\n'
6 <==> b'    <case>first-letter</case>\n'
7 <==> b'    <namespaces>\n'
8 <==> b'      <namespace key="-2" case="first-letter">Media</namespace>\n'
9 <==> b'      <namespace key="-1" case="first-letter">Special</namespace>\n'
10 <==> b'      <namespace key="0" case="first-letter" />\n'
11 <==> b'      <namespace key="1" case="first-letter">Talk</namespace>\n'
12 <==> b'      <namespace key="2" case="first-letter">User</namespace>\n'
13 <==> b'      <namespace key="3" case="first-letter">

In [25]:
df = pd.DataFrame(handler.output)
df

,0,1,2
0,David Stagg,rugby league biography,{{short description|Australian rugby league fo...
1,Amaranthus mantegazzianus,None,#REDIRECT [[Amaranthus caudatus]]\n\n{{R from ...
2,Amaranthus quitensis,None,#redirect [[Amaranthus hybridus]] {{R from tax...
3,KTXT,None,"'''KTXT''' may refer to:\n\n* [[KTXT-FM]], a r..."
4,Maud Queen of Norway,None,#REDIRECT [[Maud of Wales]]
...,...,...,...
9996,Lalla Asma,None,#REDIRECT [[Princess Lalla Asma of Morocco]]
9997,Lunar: Strolling School,None,#REDIRECT [[Lunar: Sanposuru Gakuen]]
9998,"Stapleford, Leicestershire",UK place,{{Use dmy dates|date=July 2015}}\n{{Use Britis...
9999,Rise of the Phoenix,video game,{{hatnote|For the 2014 Austrian Eurovision Son...


In [26]:
df[1].unique()

array(['rugby league biography', None, 'NRHP', 'person',
       'medical condition (new)', 'Italian comune', 'sports season',
       'NFL player', 'school', 'character', 'officeholder', 'comic',
       'UK place', 'song', 'military unit', 'medical person', 'bridge',
       'Basketball club', 'civilian attack', 'Criminal organization',
       'film', 'river', 'musical artist', 'settlement', 'monastery',
       'software', 'valley', 'Officeholder', 'Historic building',
       'webcomic', 'company', 'philosopher', 'basketball club',
       'SCOTUS case', 'comics creator', 'RPG', 'Korean name', 'cricketer',
       'building', 'LCS', 'tunnel', 'album', 'football biography',
       'NVC community', 'artist', 'prepared food', 'Australian place',
       'cell', 'television station', 'baseball biography',
       'military person', 'soap character', 'medical condition',
       'military conflict', 'venue', 'peer', 'radio station',
       'Football biography', 'clergy', 'university',
       'nati

In [27]:
df[1].value_counts()

person                180
settlement            147
French commune        127
album                 121
airport               100
                     ... 
law firm                1
cattle breed            1
religious building      1
senator                 1
                        1
Name: 1, Length: 358, dtype: int64